# Notebook using SectorPhi from SasView
## Overview
<div class="alert alert-block alert-info"> <b>NOTE</b> This notebook shortly describes how to use `sectorPhi` to calculate azimuthal average of a 2D pattern. </br> First the sign convention for the azimuthal angle is defined. Then an example using a 2D example pattern is given. The last section illustrates how `sectionPhi` works using different simpler test patterns. </div>

## Installations 
Please refer to `https://github.com/SasView/sasdata` for installation instructions.
In addition, for this notebook, you'll have to install `matplotlib` using:

In [ ]:
!python -m pip install matplotlib

## Load libraries

In [ ]:
from sasdata.dataloader.loader import Loader
import matplotlib.pyplot as plt
import numpy as np

from sasdata.dataloader.data_info import plottable_2D
from sasdata.data_util.manipulations import SectorPhi
import os

from matplotlib.colors import LogNorm
import matplotlib as mpl

plt.rcParams.update({'font.size': 16})

## Sign convention
The figure below shows the convention of origin and orientation for the azimuthal angle $\phi$ used in SasView.

<img src="./notebook_files/Fig_sasview_phi_sign_convention.png" width="300" height="300">

## Load data

The input datafile is part of the SasView test suite.

In [ ]:
data_run = Loader().load("./notebook_files/P123_D2O_10_percent.dat")

qx_data = data_run[0].qx_data
qy_data = data_run[0].qy_data
data = data_run[0].data
q_data = np.sqrt(qx_data**2 + qy_data**2)
name = os.path.splitext(os.path.basename(data_run[0].filename))[0]

## Define sector

In [ ]:
# number of sectors for phi between 0 and 2Pi
nbins_phi = 6

# minimum and maximum of ||q|| range
r_min = 0.01
r_max = 0.7 * np.sqrt(qx_data.max()**2 + qy_data.max()**2)

sect = SectorPhi(r_min=r_min, 
                 r_max=r_max,
                 phi_min=0,
                 phi_max=2 * np.pi,
                 nbins=nbins_phi)

print(f"Boundaries for r (Angstrom) and phi (rad):\nr_max={sect.r_max}, r_min={sect.r_min}, phi_max={sect.phi_max}, phi_min={sect.phi_min}.")

## Visualise azimuthal sectors
Plot of the input 2D pattern, the different azimuthal sectors (in dashed blue) used to average the data and the radial boundaries (dashed orange circles).

In [ ]:
qx_data_1d = data_run[0].x_bins
qy_data_1d = data_run[0].y_bins

nb_points_x = len(qx_data_1d)
nb_points_y = len(qy_data_1d)

lines_sectors = np.zeros((sect.nbins, 2, 2))
norm_max_q_1d = np.sqrt(np.max(qx_data_1d)**2 + np.max(qy_data_1d**2))
phi_range_per_bin = (sect.phi_max - sect.phi_min) / sect.nbins

for i in range(sect.nbins):
    # the additional np.pi is because phi=0 is pointing along the negative qx axis
    lines_sectors[i, 0, 1] = np.cos(phi_range_per_bin * i + np.pi) * norm_max_q_1d
    lines_sectors[i, 1, 1] = np.sin(phi_range_per_bin * i + np.pi) * norm_max_q_1d

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

c = plt.pcolormesh(
    qx_data_1d,
    qy_data_1d,
    data.reshape(nb_points_x, nb_points_y),
    shading='auto',
    norm=LogNorm())

# plot sectors' limits
for i in range(sect.nbins):
    ax.plot(lines_sectors[i,0,:], lines_sectors[i,1,:], 'c--')
    
# plot rmin, rmax
a_circle_min = plt.Circle(
    (0, 0), 
    r_min, 
    fill=False, 
    edgecolor="orange", 
    linestyle="--")

ax.add_artist(a_circle_min)

a_circle_max = plt.Circle(
    (0, 0), 
    r_max, 
    fill=False, 
    edgecolor="orange", 
    linestyle="--")

ax.add_artist(a_circle_max)

ax.grid()
ax.set_aspect('equal', 'box')
ax.set_xlim((np.min(qx_data_1d), np.max(qx_data_1d)))
ax.set_ylim((np.min(qy_data_1d), np.max(qy_data_1d)))
ax.set_xticks(np.linspace(-0.1, 0.1, 5))
ax.set_yticks(np.linspace(-0.1, 0.1, 5))
ax.set(title=name, xlabel=r'$q_x$', ylabel=r'$q_y$')
cbar = plt.colorbar(c, label='Intensity (arbitrary units)');

## Calculate average

In [ ]:
# Preliminary step: make the data plottable in order to be able to use SectorPhi
data_plottable = plottable_2D(
    data=data,
    err_data=np.zeros(data.shape),
    qx_data=qx_data,
    qy_data=qy_data, 
    q_data=q_data)

sector = sect(data_plottable)

## Plot result

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.plot(np.rad2deg(sector.x), sector.y, 'o-')
ax.set(title=name,
       xlabel=r'$\phi$ (degrees)',
       ylabel='Integrated intensity (arbitrary units)')
ax.grid();

## Tests of `sectorPhi` with simple 2D cases

For the following tests, we only consider the full $\phi$-range = 0, 2 $\pi$ divided into 4 sectors.  
We want to check the correctness of the average calculated in all $\phi$ sectors using different simple 2D cases.

### Settings for the $q$-vectors

In [ ]:
nbins = 150

# 1D q vectors along x and y-axes
qx_data_1d = np.linspace(-0.149, 0.149, nbins)

qy_data_1d = qx_data_1d

#2D q vectors: along x and y-axes and ||q||
qx_data = []
qy_data = []

for i in range(nbins):
    qx_data.append(qx_data_1d)
    qy_data.append(nbins*[qx_data_1d[i]])

qx_data = np.asarray(qx_data)
qy_data = np.asarray(qy_data)
q_data = np.sqrt(qx_data**2 + qy_data**2)

nb_points_x = len(qx_data_1d)
nb_points_y = len(qy_data_1d)

qx_data = qx_data.flatten()
qy_data = qy_data.flatten()

### Settings for `SectorPhi`

In [ ]:
# number of sectors
nbins_phi = 4

# we want to consider all data
r_min = 0.0

# large r_max to consider all points of the input matrix
r_max = 2 * np.sqrt(qx_data.max()**2 + qy_data.max()**2)

sect = SectorPhi(r_min=r_min, 
                 r_max=r_max,
                 phi_min=0,
                 phi_max=2 * np.pi,
                 nbins=nbins_phi)

### Definitions of the borders of the azimuthal sectors
These lines are used to plot over the 2D image to check the settings.

In [ ]:
lines_sectors = np.zeros((sect.nbins, 2, 2))
norm_max_q_1d = np.sqrt(np.max(qx_data_1d)**2 + np.max(qy_data_1d**2))
phi_range_per_bin = (sect.phi_max - sect.phi_min) / sect.nbins

for i in range(sect.nbins):
    # the additional np.pi is because phi=0 is pointing along the negative qx axis
    lines_sectors[i, 0, 1] = np.cos(phi_range_per_bin * i + np.pi) * norm_max_q_1d
    lines_sectors[i, 1, 1] = np.sin(phi_range_per_bin * i + np.pi) * norm_max_q_1d

In [ ]:
def plot_settings(data, qx_data_1d, qy_data_1d, title_plot):
    """
    Plot 2D image with layout of azimuthal sectors and radial 
    q-area to be used for averaging
    
    Parameters
    ----------
    data: 2D numpy array
        data to be plotted
    
    qx_data_1d: 1D array
        data to define the x-axis
    
    qy_data_1d: 1D array
        data to define the y-axis
    
    title_plot: str
        title to add to the figure

    Returns
    -------
    None
    """
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_aspect('equal', 'box')
    nb_points_x = len(qx_data_1d)
    nb_points_y = len(qy_data_1d)
    
    c = plt.pcolormesh(
        qx_data_1d,
        qy_data_1d,
        data.reshape(nb_points_x, nb_points_y),
        shading='auto',
        norm=LogNorm())
    
    # plot sectors' limits
    for i in range(sect.nbins):
        ax.plot(lines_sectors[i,0,:], lines_sectors[i,1,:], 'c--')
    
    # plot rmin, rmax
    a_circle_min = plt.Circle(
        (0, 0),
        r_min,
        fill=False,
        edgecolor="orange",
        linestyle="--")

    ax.add_artist(a_circle_min)

    a_circle_max = plt.Circle(
        (0, 0), 
        r_max, 
        fill=False, 
        edgecolor="orange", 
        linestyle="--")

    ax.add_artist(a_circle_max)

    ax.grid()
    ax.set_xlim((np.min(qx_data_1d), np.max(qx_data_1d)))
    ax.set_ylim((np.min(qy_data_1d), np.max(qy_data_1d)))
    ax.set(title=title_plot,
           xlabel=r'$q_x$',
           ylabel=r'$q_y$')
    cbar = plt.colorbar(c);  


def plot_result(sector_phi, title_plot):
    """
    Plot azimuthal average as function of phi in degrees
    
    Parameters
    ----------
    sector_phi : sasdata.dataloader.data_info.Data1D
        output of `sasdata.data_util.manipulations.SectorPhi`
    
    title_plot : str
        title to be added to the figure
        
    Returns
    -------
    None
    
    """
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.plot(np.rad2deg(sector_phi.x), sector_phi.y, 'o-')
    ax.grid()
    ax.set(title=title_plot, 
           xlabel=r'$\phi$ (degrees)', 
           ylabel='Integrated intensity (arbitrary units)')

## Case 1: Matrix of 1s

The square matrix is filled with 1.

In [ ]:
data_ones = np.ones((nbins, nbins)).flatten()

In [ ]:
plot_settings(data_ones, qx_data_1d, qy_data_1d, 'Matrix of 1s')

In [ ]:
# Preliminary step: make the data plottable in order to be able to use SectorPhi
data_plottable_ones= plottable_2D(
    data=data_ones,
    err_data=np.zeros(data_ones.shape),
    qx_data=qx_data,
    qy_data=qy_data, 
    q_data=np.sqrt(qx_data**2 + qy_data**2))

sector_ones = sect(data_plottable_ones)

plot_result(sector_ones, 'Matrix of 1s')

Since the input pattern was made of only ones, each of the 4 average sectors should be equal to one:

In [ ]:
print(f"Output array of the 4 calculated average sectors: {sector_ones.y}.")

## Case 2: Disk of 1s centered at 0, 0

In [ ]:
data_circle = np.zeros((nbins, nbins))

# counts per sectors 

count_mm = 0  # q_x<0, q_y<0
count_mp = 0  # q_x<0, q_y>0
count_pm = 0  # q_x>0, q_y<0
count_pp = 0  # q_x>0, q_y>0

for i in range(nbins):
    for j in range(nbins):   
        if np.sqrt(qx_data_1d[i]**2 + qy_data_1d[j]**2) <= 0.04:
            data_circle[i, j] = 1
            # count ones in each sector
            if qx_data_1d[i]>0:
                if qy_data_1d[j]>0:
                    count_pp += 1
                else:
                    count_pm += 1
            else:
                if qy_data_1d[j]>0:
                    count_mp += 1
                else:
                    count_mm += 1  
            
data_circle = data_circle.flatten()

In [ ]:
# check that the sum of all 4 counts corresponds to the number of ones
print((f"The number of '1's in the input data, i.e., {np.count_nonzero(data_circle==1)}" 
       f" should be equal to the sum counted in each sector, i.e., {count_mm + count_pp + count_mp + count_pm}."))

In [ ]:
plot_settings(data_circle, qx_data_1d, qy_data_1d, 'Disk of 1s')

In [ ]:
# Preliminary step: make the data plottable in order to be able to use SectorPhi
data_plottable_c = plottable_2D(
    data=data_circle,
    err_data=np.zeros(data_circle.shape),
    qx_data=qx_data,
    qy_data=qy_data, 
    q_data=np.sqrt(qx_data**2 + qy_data**2))

sector_circle = sect(data_plottable_c)

plot_result(sector_circle, 'Disk')

For this input disk pattern, the average value in each sector should be all be equal. They should correspond to the number of counts divided by the number of bins per sector.  

The number of counts per sector, `count_mm`, `count_mp`, `count_pm` and `count_pp`, have been calculated above.  

The number of bins per sector is equal to the size of the 2D grid divided by the number of sectors, *i.e.*, nbins$^2$/nbins_phi = $150^2/4$.

In [ ]:
# check that the azimuthal average corresponds to the number of ones per sector divided 
# by the size of the sector.

nbins_per_sect = (nbins**2/nbins_phi)

print((f"Number of ones per sector divided by the size of the sector:\n"
       f"region 'q_x<0, q_y<0': {count_mm/nbins_per_sect:.8f},\nregion 'q_x<0, q_y>0': {count_mp/nbins_per_sect:.8f},\n"
       f"region 'q_x>0, q_y>0': {count_pp/nbins_per_sect:.8f},\nregion 'q_x>0, q_y<0': {count_pm/nbins_per_sect:.8f}\n"
       f"Output array of the 4 calculated average sectors: {sector_circle.y}"
     ))

## Case 3: circle centered at 0, 0 with zeros at the sectors' edges

In [ ]:
data_circle0 = np.zeros((nbins, nbins))

# counts per sectors 

count_mm = 0  # q_x<0, q_y<0
count_mp = 0  # q_x<0, q_y>0
count_pm = 0  # q_x>0, q_y<0
count_pp = 0  # q_x>0, q_y>0

for i in range(nbins):
    for j in range(nbins):
        if np.sqrt((i - nbins//2)**2 + (j - nbins//2)**2) < 20:
            data_circle0[i, j] = 1
            
            if qx_data_1d[i]>0:
                if qy_data_1d[j]>0:
                    count_pp += 1
                else:
                    count_pm += 1      
            else: 
                if qy_data_1d[j]>0:
                    count_mp += 1
                else:
                    count_mm += 1  
            
            if i-nbins//2 == 0 or j-nbins//2 == 0:   
                data_circle0[i, j] = 0
                if qx_data_1d[i]>0:
                    if qy_data_1d[j]>0:
                        count_pp -= 1
                    else: 
                        count_pm -= 1    
                else: 
                    if qy_data_1d[j]>0:
                        count_mp -= 1
                    else:
                        count_mm -= 1  

data_circle0 = data_circle0.flatten()

In [ ]:
# check that the sum of all 4 counts corresponds to the number of ones
print((f"The number of '1's in the input data, i.e., {np.count_nonzero(data_circle0==1)}" 
       f" should be equal to the sum counted in each sector, i.e., {count_mm + count_pp + count_mp + count_pm}."))

In [ ]:
plot_settings(data_circle0, qx_data_1d, qy_data_1d, 'Disk of 1s with 0s on axes')

In [ ]:
# Preliminary step: make the data plottable in order to be able to use SectorPhi
data_plottable_c0 = plottable_2D(
    data=data_circle0,
    err_data=np.zeros(data_circle0.shape),
    qx_data=qx_data,
    qy_data=qy_data, 
    q_data=np.sqrt(qx_data**2+qy_data**2)
)

sector_circle0 = sect(data_plottable_c0)

plot_result(sector_circle0, 'Matrix of 1s')

For this input pattern, the average value in each sector should be all be equal. They should correspond to the number of counts divided by the number of bins per sector.  

The number of counts per sector, `count_mm`, `count_mp`, `count_pm` and `count_pp`, have been calculated above.  

The number of bins per sector is equal to the size of the 2D grid divided by the number of sectors, *i.e.*, nbins$^2$/nbins_phi = $150^2/4$.

In [ ]:
# check that the azimuthal average corresponds to the number of ones per sector divided 
# by the size of the sector.
nbins_per_sect = (nbins**2/nbins_phi)
print((f"Number of ones per sector divided by the size of the sector:\n"
       f"region 'q_x<0, q_y<0': {count_mm/nbins_per_sect:.8f},\nregion 'q_x<0, q_y>0': {count_mp/nbins_per_sect:.8f},\n"
       f"region 'q_x>0, q_y>0': {count_pp/nbins_per_sect:.8f},\nregion 'q_x>0, q_y<0': {count_pm/nbins_per_sect:.8f}\n"
       f"Output array of the 4 calculated average sectors: {sector_circle0.y}"
     ))

## Case 4: small square of 1s

Square pattern of 1s, smaller than the whole grid

In [ ]:
data_square = np.zeros((nbins, nbins))

# counts per sectors 
count_mm = 0  # q_x<0, q_y<0
count_mp = 0  # q_x<0, q_y>0
count_pm = 0  # q_x>0, q_y<0
count_pp = 0  # q_x>0, q_y>0

for i in range(nbins):
    for j in range(nbins):
        if np.abs(qx_data_1d[i]) <= 0.04 and np.abs(qy_data_1d[j]) <= 0.04:
            data_square[i, j] = 1
            # count ones in each sector
            if qx_data_1d[i]>0:
                if qy_data_1d[j]>0:
                    count_pp += 1
                else:
                    count_pm += 1
            else:
                if qy_data_1d[j]>0:
                    count_mp += 1
                else:
                    count_mm += 1
            
data_square = data_square.flatten()

In [ ]:
# check that the sum of all 4 counts corresponds to the number of ones
print((f"The number of '1's in the input data, i.e., {np.count_nonzero(data_square==1)}" 
       f" should be equal to the sum counted in each sector, i.e., {count_mm + count_pp + count_mp + count_pm}."))

In [ ]:
plot_settings(data_square, qx_data_1d, qy_data_1d, 'Square of 1s')

In [ ]:
# Preliminary step: make the data plottable in order to be able to use SectorPhi
data_plottable_sq = plottable_2D(
    data=data_square,
    err_data=np.zeros(data_square.shape),
    qx_data=qx_data,
    qy_data=qy_data, 
    q_data=np.sqrt(qx_data**2 + qy_data**2)
)

sector_sq = sect(data_plottable_sq)

plot_result(sector_sq, 'Square of 1s')

In [ ]:
# check that the azimuthal average corresponds to the number of ones per sector divided 
# by the size of the sector.

nbins_per_sect = (nbins**2/nbins_phi)
print((f"Number of ones per sector divided by the size of the sector:\n"
       f"region 'q_x<0, q_y<0': {count_mm/nbins_per_sect:.8f},\nregion 'q_x<0, q_y>0': {count_mp/nbins_per_sect:.8f},\n"
       f"region 'q_x>0, q_y>0': {count_pp/nbins_per_sect:.8f},\nregion 'q_x>0, q_y<0': {count_pm/nbins_per_sect:.8f}\n"
       f"Output array of the 4 calculated average sectors: {sector_sq.y}"
     ))

## Case 5: small square of 1s and 0s on the axes

In [ ]:
data_square0 = np.zeros((nbins, nbins))

# counts per sectors 
count_mm = 0  # q_x<0, q_y<0
count_mp = 0  # q_x<0, q_y>0
count_pm = 0  # q_x>0, q_y>0
count_pp = 0  # q_x>0, q_y>0

for i in range(nbins):
    for j in range(nbins):
        if 0 < abs(i-nbins//2) < 20 and 0 < abs (j-nbins//2) < 20:
            data_square0[i, j] = 1
            # count ones in each sector
            if qx_data_1d[i]>0:
                if qy_data_1d[j]>0:
                    count_pp += 1
                else:
                    count_pm += 1
            else:
                if qy_data_1d[j]>0:
                    count_mp += 1
                else:
                    count_mm += 1  

data_square0 = data_square0.flatten()

In [ ]:
# check that the sum of all 4 counts corresponds to the number of ones
print((f"The number of '1's in the input data, i.e., {np.count_nonzero(data_square0==1)}" 
       f" should be equal to the sum counted in each sector, i.e., {count_mm + count_pp + count_mp + count_pm}."))

In [ ]:
plot_settings(data_square0, qx_data_1d, qy_data_1d, 'Square of 1s with 0s on axes')

In [ ]:
# Preliminary step: make the data plottable in order to be able to use SectorPhi
data_plottable_sq0 = plottable_2D(
    data=data_square0,
    err_data=np.zeros(data_square0.shape),
    qx_data=qx_data,
    qy_data=qy_data, 
    q_data=np.sqrt(qx_data**2 + qy_data**2)
)

sector_sq0 = sect(data_plottable_sq0)

plot_result(sector_sq0, 'Square0')

In [ ]:
# check that the azimuthal average corresponds to the number of ones per sector divided 
# by the size of the sector.
nbins_per_sect = (nbins**2/nbins_phi)
print((f"Number of ones per sector divided by the size of the sector:\n"
       f"region 'q_x<0, q_y<0': {count_mm/nbins_per_sect:.8f},\nregion 'q_x<0, q_y>0': {count_mp/nbins_per_sect:.8f},\n"
       f"region 'q_x>0, q_y>0': {count_pp/nbins_per_sect:.8f},\nregion 'q_x>0, q_y<0': {count_pm/nbins_per_sect:.8f}\n"
       f"Output array of the 4 calculated average sectors: {sector_sq0.y}"
     ))

## Case 6: quadrant with different constant values in each sector

In [ ]:
data_quadrant = np.zeros((nbins, nbins))

# counts per sectors 
# (the index marks the area containing this value
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0

for i in range(nbins):
    for j in range(nbins):
        if i < nbins//2:
            if j < nbins//2:
                data_quadrant[i, j] = 3
                count_3 += 1
            else:
                data_quadrant[i, j] = 4
                count_4 += 1
        else:
            if j < nbins//2:
                data_quadrant[i, j] = 2
                count_2 += 1
            else:
                data_quadrant[i, j] = 1
                count_1 += 1

data_quadrant = data_quadrant.flatten()

In [ ]:
# check that all nonzeros values in each sector have been counted:
print((f"The number of '1's in the input data in sector 1, i.e., {np.count_nonzero(data_quadrant == 1)}" 
       f" should be equal to the count in sector 1, i.e., {count_1}.\n"
       f"The number of '2's in the input data in sector 2, i.e., {np.count_nonzero(data_quadrant == 2)}" 
       f" should be equal to the count in sector 2, i.e., {count_2}.\n"
       f"The number of '3's in the input data in sector 3, i.e., {np.count_nonzero(data_quadrant == 3)}" 
       f" should be equal to the count in sector 3, i.e., {count_3}.\n"
       f"The number of '4's in the input data in sector 4, i.e., {np.count_nonzero(data_quadrant == 4)}" 
       f" should be equal to the count in sector 4, i.e., {count_4}.\n" 
      ))

In [ ]:
fig, ax = plt.subplots()

cmap = mpl.colors.ListedColormap(['#2c7bb6','#e389b9', '#746ab0', '#6c5b7b'])
norm = mpl.colors.BoundaryNorm([0.5, 1.5, 2.5, 3.5, 4.5], cmap.N)

c = plt.pcolormesh(
    qx_data_1d, 
    qy_data_1d, 
    data_quadrant.reshape(nb_points_x, nb_points_y),
    shading='auto',
    cmap=cmap,
    norm=norm)

# plot sectors' limits
for i in range(sect.nbins):
    ax.plot(lines_sectors[i,0,:], lines_sectors[i,1,:], 'c--')
    
# plot rmin, rmax
a_circle_min = plt.Circle((0, 0), r_min, fill=False, edgecolor="orange", linestyle="--")
ax.add_artist(a_circle_min)

a_circle_max = plt.Circle((0, 0), r_max, fill=False, edgecolor="orange", linestyle="--")
ax.add_artist(a_circle_max)

ax.grid()
ax.set_xlim((np.min(qx_data_1d), np.max(qx_data_1d)))
ax.set_ylim((np.min(qy_data_1d), np.max(qy_data_1d)))
ax.set(title='Quadrant',
       xlabel=r'$q_x$',
       ylabel=r'$q_y$')
cbar = plt.colorbar(c, ticks=[1, 2, 3, 4], format='%d');

In [ ]:
# Preliminary step: make the data plottable in order to be able to use SectorPhi
data_plottable_q = plottable_2D(
    data=data_quadrant,
    err_data=np.zeros(data_quadrant.shape),
    qx_data=qx_data,
    qy_data=qy_data, 
    q_data=np.sqrt(qx_data**2 + qy_data**2))

sector_quadrant = sect(data_plottable_q)

plot_result(sector_quadrant, 'Quadrant')

Each calculated average value per sector should either be equal to 1, 2, 3 or 4. 
Considering to orientation of Phi and the definition of the input matrix, the array of calculated value should be [3, 4, 1, 2]:

In [ ]:
sector_quadrant.y